<a href="https://colab.research.google.com/github/seunghuilee91/Deeplearning/blob/master/Decision_Tree_%26_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append("/content/gdrive/My Drive/Colab Notebooks")


# Decision Tree (결정트리, 의사결정나무)

## 모델 소개
---

<br>

- 의사결정나무 학습법(Decision tree learning)은 어떤 항목에 대한 관측값과 목표값을 연결시켜주는 예측 모델로써 결정 트리를 사용한다. <br>
 *스무고개 놀이의  개념*

- **분류와 회귀** 모두 가능한 모델이다. 즉 범주나 연속형 수치 모두 예측 가능하다. (분류나무, 회귀나무로 나뉜다)

- 분류의 경우 최빈값, 회귀의 경우 종속변수(y)의 평균을 예측값으로 반환한다.

<br>
<br>

![의사결정나무 예시](https://miro.medium.com/max/1380/1*xzF10JmR3K0rnZ8jtIHI_g.png)





## 불순도 / 불확실성
---

<br>

- 분류나무는 구분 뒤 각 영역의 순도(homogeneity)가 증가, 불순도(impurity) 혹은 불확실성(uncertainty)이 최대한 감소하도록 하는 방향으로 학습을 진행한다.

- 순도가 증가/불확실성이 감소하는 걸 두고 정보이론에서는 **정보획득**(information gain)이라고 한다.
 <br> <br>
#### 순도를 계산하는 3가지 방식
 1. 엔트로피(Entropy)

 2. 지니계수(Gini Index)

 2. 오분류오차(Misclassification Error) <br>
  '불순도를 측정할 수 있긴 하나 위 두 지표와 달리 미분이 불가능한 점 때문에 자주 쓰이지 않음 


<br>
<br>


> **엔트로피**(entropy) <br>
*얼마만큼의 정보를 담고 있는가*


- 분포 p를 가진 H(X)로 표기되는 확률변수의 정보 
-  **불확실성의 측정** 으로 기억!
- 즉  확률 값임으로 0 ~ 1 사이의 실수를 가지며, 늘 양수임

엔트로피가 '0'이라는 말은 현상이 발생할 확률이 0% 라는 것이다. 즉, 사건의 분포가 결정적(deterministic)이라면 해당 확률분포의 불확실성 정도를 나타내는 엔트로피는 낮아진다. 반대로 분포가 균등적(uniform)일 수록 엔트로피는 높아진다. (ex. 동전던지기, 엔트로피는 1이다. 결과값을 예상하기가 어렵기 때문이다.(불확실성 최대치)

In [0]:
from typing import List
import math

def entropy(class_probabilities: List[float]) -> float:
    """Given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2)
               for p in class_probabilities
               if p > 0)                     # ignore zero probabilities

assert entropy([1.0]) == 0
assert entropy([0.5, 0.5]) == 1
assert 0.81 < entropy([0.25, 0.75]) < 0.82

from typing import Any
from collections import Counter

def class_probabilities(labels: List[Any]) -> List[float]:
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities(labels))

assert data_entropy(['a']) == 0
assert data_entropy([True, False]) == 1
assert data_entropy([3, 4, 4, 4]) == entropy([0.25, 0.75])

def partition_entropy(subsets: List[List[Any]]) -> float:
    """Returns the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    return sum(data_entropy(subset) * len(subset) / total_count
               for subset in subsets)

from typing import NamedTuple, Optional

class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None  # allow unlabeled data

                  #  level     lang     tweets  phd  did_well
inputs = [Candidate('Senior', 'Java',   False, False, False),
          Candidate('Senior', 'Java',   False, True,  False),
          Candidate('Mid',    'Python', False, False, True),
          Candidate('Junior', 'Python', False, False, True),
          Candidate('Junior', 'R',      True,  False, True),
          Candidate('Junior', 'R',      True,  True,  False),
          Candidate('Mid',    'R',      True,  True,  True),
          Candidate('Senior', 'Python', False, False, False),
          Candidate('Senior', 'R',      True,  False, True),
          Candidate('Junior', 'Python', True,  False, True),
          Candidate('Senior', 'Python', True,  True,  True),
          Candidate('Mid',    'Python', False, True,  True),
          Candidate('Mid',    'Java',   True,  False, True),
          Candidate('Junior', 'Python', False, True,  False)
         ]

from typing import Dict, TypeVar
from collections import defaultdict

T = TypeVar('T')  # generic type for inputs


In [0]:
# 1 단계 : 가장 낮은 엔트로피를 반환하는 파티션 찾기

def partition_by(inputs: List[T], attribute: str) -> Dict[Any, List[T]]:
    """Partition the inputs into lists based on the specified attribute."""
    partitions: Dict[Any, List[T]] = defaultdict(list)
    for input in inputs:
        key = getattr(input, attribute)  # value of the specified attribute
        partitions[key].append(input)    # add input to the correct partition
    return partitions


In [0]:
# 2 단계 : 엔트로피를 계산
def partition_entropy_by(inputs: List[Any],
                         attribute: str,
                         label_attribute: str) -> float:
    """Compute the entropy corresponding to the given partition"""
    # partitions consist of our inputs
    partitions = partition_by(inputs, attribute)

    # but partition_entropy needs just the class labels
    labels = [[getattr(input, label_attribute) for input in partition]
              for partition in partitions.values()]

    return partition_entropy(labels)


In [0]:

# 3 단계 : 전체 데이터셋에 대해 엔트로피를 최소화하는 판티션 찾기
for key in ['level','lang','tweets','phd']:
    print(key, partition_entropy_by(inputs, key, 'did_well'))

# 직급에 대해 파티션을 나눌 때 엔트로피가 최솟값을 가진다. 

In [0]:
from collections import Counter, defaultdict
from functools import partial
import math, random

def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    return sum( data_entropy(subset) * len(subset) / total_count
                for subset in subsets )

def group_by(items, key_fn):
    """returns a defaultdict(list), where each input item
    is in the list whose key is key_fn(item)"""
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups

def partition_by(inputs, attribute):
    """returns a dict of inputs partitioned by the attribute
    each input is a pair (attribute_dict, label)"""
    return group_by(inputs, lambda x: x[0][attribute])

def partition_entropy_by(inputs,attribute):
    """computes the entropy corresponding to the given partition"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

In [0]:

def classify(tree, input):
    """classify the input using the given decision tree"""

    # if this is a leaf node, return its value
    if tree in [True, False]:
        return tree

    # otherwise find the correct subtree
    attribute, subtree_dict = tree

    subtree_key = input.get(attribute)  # None if input is missing attribute

    if subtree_key not in subtree_dict: # if no subtree for key,
        subtree_key = None              # we'll use the None subtree

    subtree = subtree_dict[subtree_key] # choose the appropriate subtree
    return classify(subtree, input)     # and use it to classify the input

def build_tree_id3(inputs, split_candidates=None):

    # if this is our first pass,
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()

    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues

    if num_trues == 0:                  # if only Falses are left
        return False                    # return a "False" leaf

    if num_falses == 0:                 # if only Trues are left
        return True                     # return a "True" leaf

    if not split_candidates:            # if no split candidates left
        return num_trues >= num_falses  # return the majority leaf

    # otherwise, split on the best attribute
    best_attribute = min(split_candidates,
        key=partial(partition_entropy_by, inputs))

    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    # recursively build the subtrees
    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

def forest_classify(trees, input):
    votes = [classify(tree, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]


if __name__ == "__main__":

    inputs = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
    ]

    for key in ['level','lang','tweets','phd']:
        print(key, partition_entropy_by(inputs, key))
    print()

    senior_inputs = [(input, label)
                     for input, label in inputs if input["level"] == "Senior"]

    for key in ['lang', 'tweets', 'phd']:
        print(key, partition_entropy_by(senior_inputs, key))
    print()

    print("building the tree")
    tree = build_tree_id3(inputs)
    print(tree)

    print("Junior / Java / tweets / no phd", classify(tree,
        { "level" : "Junior",
          "lang" : "Java",
          "tweets" : "yes",
          "phd" : "no"} ))

    print("Junior / Java / tweets / phd", classify(tree,
        { "level" : "Junior",
                 "lang" : "Java",
                 "tweets" : "yes",
                 "phd" : "yes"} ))

    print("Intern", classify(tree, { "level" : "Intern" } ))
    print("Senior", classify(tree, { "level" : "Senior" } ))

### - 분류나무 모델

### - 회귀나무 모델
#### 모델링 프로세스 
- 데이터를 M개로 분할 R1,R2 ...Rm
- 최상의 분할은 다음 비용함수를 최소로 할 때 얻어짐
- 각 분할에 속해 있는 y값들의 평균으로 예측했을 때 오류가 최소됨
- 분할변수(j)와 분할점(s)은 어떻게 결정할까?


## 모델 학습
---
<br>

의사결정나무의 학습 과정은 입력 변수 영역을 두 개로 구분하는 **재귀적 분기(recursive partitioning)** 와 너무 자세하게 구분된 영역을 통합하는 **가지치기(pruning)** 두 가지로 과정으로 나뉜다.

### 1) 재귀적 분기


### 2) 가지치기 (Pruning)
- Terminal node의 순도가 100%인 상태를 Full tree라고 한다. 분기가 너무 많을 경우 학습데이터에 과적합(overfitting)할 우려가 있기 때문에 적절한 수준에서 **가지치기** 가 필요하다.

- 결정나무의 분기 수가 일정한 수준 이상이 되면 오분류율이 오히려 증가하는 현상이 발생한다. 이에 검증데이터에 대한 오분류율이 증가하는 시점을 파악하여 적절히 가지치기를 수행해야 한다.

- 나뭇가지를 잘라내는 것과 같다는 의미에서 용어가 붙었으나 실제 개념은 분기를 합치는 Merge이 개념으로 이해해야 한다.

## 장단점 및 활용
---

<br>

#### 장점
- 숫자형 데이터와 범주형 데이터를 동시에 다룰 수 있음
- 특정 변수의 값이 누락되어도 사용 가능
- 계산복잡성 대비 높은 예측 성능
- 변수 단위 설명력(이해하고 해석하귀 용이함)


#### 단점
- 새로운 데이터에 대한 일반화 성능이 좋지 않아 오버피팅이 되기 쉬움( = 결정경계(decision boundary)가 데이터 축에 
수직이어서 특정 데이터에만 잘 작동할 가능성 크다!)
<br>

 ☞ *이를 극복하기 위해 **랜덤포레스트** 모델이 등장함*



[참고-코드]

In [0]:
def entropy(class_probabilities):
    """클래스에 속할 확률을 입력하면 엔트로피를 계산하라"""
    return sum(-p * math.log(p,2)
    for p in class_probabilities if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

def partition_entropy(subsets):
    total_count = sum(len(subset) for subset in subsets)
    return sum( data_entropy(subset) * len(subset / total_count
                                           for subset in subsets))

# Random Forest(랜덤 포레스트)
---

- 의사결정나무가 학습 데이터에 꼭 들어맞는다? = 오버피팅
- 오버피팅을 방지할 수 있는 대표적인 방법 중 하나로 랜덤포레스트라는 것이 등장함
- 여러 개의 의사결정나무를 만들고, 그들의 다수결로 결과를 결정하는 방법



In [0]:
def forest_classify(trees, input):
    votes = [classify(tree, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]

## 부트스트랩(Bootstrap)
- 전체 데이터가 inputs라고 했을 때 그것을 전부 이용해서 나무를 학습하는 것이 아니라 bootstrap_sample(inputs)의 결과물을 각 나무의 입력값으로 넣어 학습하는 것

- 각 나무가 서로 다른 데이터로 구축되기 때문에 랜덤성이 생기게 된다. (이점 : 샘플링되지 않는 데이터는 테스트 데이터로 이용 가능, 즉 성능을 측정하는 방법만 잘 설계한다면, 데이터 전체를 학습에 사용해도 된다는 것을 의미한다.)
